# Extract entities from initiatives websites
## The process

In [ ]:
from requests_xml import XMLSession
import requests
import htmlmin
import pandas as pd
from rdflib.graph import Graph
from itertools import groupby
from rdflib.namespace import Namespace
from bs4 import BeautifulSoup

In [212]:
def request_site(url):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
    headers = {'User-Agent': user_agent}
    request = requests.get(url, timeout=10, headers=headers)
    response = htmlmin.minify(request.text, remove_comments=True, remove_empty_space=True, reduce_boolean_attributes=True,remove_optional_attribute_quotes=True)
    response = response.encode('utf-8')
    # response = response[0:126900]
    return response

def parse_graph(request, verbose=False):
    # Initiatile a RDF graph
    graph = Graph()
    # namespace = Namespace('http://s.opencalais.com/1/type/sys/')
    # Parse data from request
    data = graph.parse(data=request.text, format="application/rdf+xml")
    # Data should be extrapolated by subject and his attributes
    entities = pd.DataFrame(columns=['subj','pred','obj'])

    # Iterate over graph entities
    # And remove urls from data
    for subj, pred, obj in data:
        s_subj = (str(subj)).replace('http://d.opencalais.com/dochash-1/', '')
        s_subj = '/'.join(s_subj.split('/')[1:])
        s_pred = (str(pred)).replace('http://www.w3.org/1999/02/22-rdf-syntax-ns#', '')
        s_pred = s_pred.replace('http://s.opencalais.com/1/pred/', '')
        s_obj = (str(obj)).replace('http://s.opencalais.com/1/type/', '')
        entities = entities.append({'subj': s_subj, 'pred': s_pred, 'obj': s_obj }, ignore_index=True)

    # Filter anything which isn't relevant for us
    entities = entities[~(entities['subj'] == 'ComponentVersions')]
    entities = entities[~(entities['subj'].str.contains('Instance'))]
    entities = entities[~(entities['subj'].str.contains('Relevance'))]
    entities = entities[~(entities['obj'].str.contains('tag/Confidence'))]
    # Group by entity subject
    grouped = entities.groupby('subj')

    # Print data for visualization
    if verbose:
        for name,group in grouped:
            print(name)
            print(group[['pred','obj']])
            print('-'*20)

    return grouped

In [193]:
permid_org_token = "Nwd6TcIxPvWPmuCvJGo48gt1e5TLyol9"
headers = {
    'X-AG-Access-Token': permid_org_token,
    'Content-Type': 'text/html'
}
data = request_site('https://tribalclimate.uoregon.edu/')

In [194]:
request = requests.post('https://api.thomsonreuters.com/permid/calais', data=data, headers=headers)

In [195]:
graph = Graph()
namespace = Namespace('http://s.opencalais.com/1/type/sys/')
data = graph.parse(data=request.text, format="application/rdf+xml")

In [201]:
entities = pd.DataFrame(columns=['subj','pred','obj'])

for subj, pred, obj in data:
    s_subj = (str(subj)).replace('http://d.opencalais.com/dochash-1/', '')
    s_subj = '/'.join(s_subj.split('/')[1:])
    s_pred = str(pred)
    #if s_pred == 'http://s.opencalais.com/1/pred/name':
    entities = entities.append({'subj': s_subj, 'pred': s_pred, 'obj': obj },ignore_index=True)

    
entities = entities[~(entities['subj'] == 'ComponentVersions')]
entities = entities[~(entities['subj'].str.contains('Instance'))]
grouped = entities.groupby('subj')

for name,group in grouped:
    print(name)
    print(group[['pred','obj']])
    print('-'*20)


                                                pred  \
64            http://s.opencalais.com/1/pred/docDate   
160   http://s.opencalais.com/1/pred/calaisRequestID   
523          http://s.opencalais.com/1/pred/docTitle   
535  http://www.w3.org/1999/02/22-rdf-syntax-ns#type   
589                http://s.opencalais.com/1/pred/id   
892          http://s.opencalais.com/1/pred/ontology   
951          http://s.opencalais.com/1/pred/document   

                                                   obj  
64                             2019-03-21 13:14:39.246  
160               89f3dd8d-26b9-a7b1-169a-063a4283992e  
523                      Tribal Climate Change Project  
535         http://s.opencalais.com/1/type/sys/DocInfo  
589    http://id.opencalais.com/2-ZZ6mOTVlTKJV9-kYH12A  
892  http://mdaas-virtual-onecalais.int.thomsonreut...  
951  <body> Home | Tribal Climate Change Project.Ab...  
--------------------
/d.opencalais.com/comphash-1/171f9df1-9c32-30ec-afb5-7b12112b39e7
       

In [210]:
data = requests.get('http://DeepWaterGroup.org')
response = BeautifulSoup(data.text)
for s in response(['script', 'style']):
    s.decompose()

In [211]:
response.get_text()

" \n\nDeepwater Group | \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n \n\n\n\nAbout\n\nWho we are\nContact\n\n\nFisheries\n\nHake (HAK)\nHoki (HOK)\nJack Mackerel (JMA)\nLing (LIN)\nOrange Roughy (ORH)\nOreo (OEO)\nScampi (SCI)\nSouthern Blue Whiting (SBW)\nSquid (SQU)\n\n\nManagement\nEnvironment\n\nDolphins\nSea Lions\nSharks\nBenthic\nBycatch\nEcosystems\nSeabirds\n\n\nCertification\n\nOrange Roughy Second Surveillance Audit 2018\nNew Zealand Hoki, Hake and Ling Trawl Fishery Re-assessment 2017\nNew Zealand Ling Longline Fishery Re-assessment 2017\nSouthern Blue Whiting Trawl Fishery Re-assessment 2017\n\n\nSeafood\n\nSeafood Recipes\n\n\nNews & Resources\n\nOperational Procedures\nOpenSeas.org.nz\n\n\n\n\nNEWSMSC validates New Zealand seafood sustainabilityNew Zealand hoki quota owners proactively choosing to reduce catchIncreased catches f